In [ ]:
pip uninstall -y qfinuwa

In [ ]:
!pip install qfinuwa

In [2]:
import pandas as pd
from qfinuwa import Indicators, Strategy, Backtester, Plotting

In [23]:
class CustomIndicators(Indicators):

    @Indicators.SingleIndicator
    def pairs_ratio(self, stocks, WINDOW_SIZE = 30):
        vgt_mid_price = (stocks['VGT']['high'] + stocks['VGT']['low']) / 2
        iyw_mid_price = (stocks['IYW']['high'] + stocks['IYW']['low']) / 2
    
        price_ratio = vgt_mid_price / iyw_mid_price

        rolling_mean = price_ratio.rolling(window=WINDOW_SIZE).mean()
        rolling_std = price_ratio.rolling(window=WINDOW_SIZE).std()
        z_score = (price_ratio - rolling_mean) / rolling_std

        #calculates cmf
        mfm = ((data['close'] - data['low']) - (data['high'] - data['close'])) / (data['high'] - data['low'])
        mf_volume = mfm * data['volume']
        cmf = mf_volume.rolling(window=period).sum() / data['volume'].rolling(window=period).sum()
        return {"z_score" : z_score, "cmf" : cmf}
    
    @Indicators.SingleIndicator
    def cmf(self, stocks, stock, WINDOW_SIZE=30, CMF_PERIOD=20):
        stock_mid_price = (stocks[stock]['high'] + stocks[stock]['low']) / 2


        # calculate CMF
        mfm = ((2 * stock_mid_price - stocks[stock]['low'] - stocks[stock]['high']) / (stocks[stock]['high'] - stocks[stock]['low']))
        mf_volume = mfm * stocks[stock]['volume']
        cmf = mf_volume.rolling(window=CMF_PERIOD).sum() / stocks[stock]['volume'].rolling(window=CMF_PERIOD).sum()

        return {"cmf": cmf}
    
    @Indicators.SingleIndicator
    def rsi(self, stocks, stock, WINDOW_SIZE=14):
        # create mid prices variable
        prices = (stocks[stock]['high'] + stocks[stock]['low']) / 2
        #calculate the period 
        period = WINDOW_SIZE
        gains = []
        losses = []
        for i in range(1, len(prices)):
            diff = prices[i] - prices[i - 1]
            if diff >= 0:
                gains.append(diff)
                losses.append(0)
            else:
                gains.append(0)
                losses.append(abs(diff))

        def simple_moving_average(prices, window):
            return [sum(prices[i:i + window]) / window for i in range(len(prices) - window + 1)]
        avg_gains = simple_moving_average(gains, period)
        avg_losses = simple_moving_average(losses, period)

        rsi = [g / l if l != 0 else 0 for g, l in zip(avg_gains, avg_losses)]
        return {"rsi": rsi}
   

In [16]:
ci = CustomIndicators()
data = ['VGT', 'IYW']
stocks = {d: pd.read_csv('../data/' + d + '.csv') for d in data}
z_score = ci.pairs_ratio(stocks)

In [18]:
class PairsTrading(Strategy):

    def __init__(self, quantity=5):
        self.quantity = quantity
        self.vgt_position = 0
        self.iyw_position = 0

    def on_data(self, prices, indicators, portfolio): 
        # On average, VGT is 4.05 times more expensive than IYW

        # If z_score is greater than 3, buy VGT and sell IYW
        if indicators['z_score'][-1] > 3 and abs(self.vgt_position < 250):
            portfolio.order('VGT', quantity = self.quantity)
            portfolio.order('IYW', quantity = -4*self.quantity)
            self.vgt_position += self.quantity
            self.iyw_position -= self.quantity
        
        # If z_score is greater than 1.5, buy VGT and sell IYW to half delta limit
        if indicators['z_score'][-1] > 1.5 and abs(self.vgt_position < 125) and abs(self.iyw_position < 500):
            portfolio.order('VGT', quantity = self.quantity)
            portfolio.order('IYW', quantity = -4*self.quantity)
            self.vgt_position += self.quantity
            self.iyw_position -= self.quantity
        
        # If z_score is less than -3, buy IYW and sell VGT
        elif indicators['z_score'][-1] < -3 and abs(self.vgt_position < 250):
            portfolio.order('VGT', quantity = -self.quantity)
            portfolio.order('IYW', quantity = 4*self.quantity)
            self.vgt_position -= self.quantity
            self.iyw_position += self.quantity

        # If z_score is less than -1.5, buy IYW and sell VGT to half delta limit
        elif indicators['z_score'][-1] < -1.5 and abs(self.vgt_position < 125) and abs(self.iyw_position < 500):
            portfolio.order('VGT', quantity = -self.quantity)
            portfolio.order('IYW', quantity = 4*self.quantity)
            self.vgt_position -= self.quantity
            self.iyw_position += self.quantity
        
        # If z_score is between -0.5 and 0.5, exit positions
        elif abs(indicators['z_score'][-1]) < 0.5:
            portfolio.order('VGT', quantity = -self.vgt_position)
            portfolio.order('IYW', quantity = -self.iyw_position)
        return
    
    def on_finish(self):
        return self.vgt_position, self.iyw_position
        

In [24]:
backtester = Backtester(PairsTrading, CustomIndicators, ['VGT', 'IYW'], data_folder='../data', days=30, fee=0.000, delta_limits=1000)
backtester

> Precompiling data: 100%|██████████| 431430/431430 [00:04<00:00, 88572.36it/s]


Backtester:
- Strategy: PairsTrading
	- Params: {'quantity': 5}
- Indicators: CustomIndicators
	- Params: {'pairs_ratio': {'WINDOW_SIZE': 30}}
	- SingleIndicators: ['z_score']
	- MultiIndicators: []
- Stocks: ['IYW', 'VGT']
- Fee 0.0
- delta_limit: {'IYW': 1000, 'VGT': 1000}
- Days: 30

In [22]:
backtester.run(cv=20)

> Running backtest over 20 samples of 30 days: 100%|██████████| 20/20 [00:17<00:00,  1.15it/s]



{'strategy': {'quantity': 5}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 50}, 'pairs_ratio': {'WINDOW_SIZE': 30}}}

Mean ROI:	177822.08115587564
STD ROI:	108159.84676288172

24/09/2021 -> 22/10/2021:	317491.111
17/08/2021 -> 15/09/2021:	216076.202
10/11/2021 -> 09/12/2021:	218087.453
20/09/2022 -> 19/10/2022:	-4060.716
29/03/2022 -> 27/04/2022:	-27283.747
28/02/2022 -> 29/03/2022:	184838.585
04/11/2021 -> 03/12/2021:	302253.151
21/07/2022 -> 19/08/2022:	183110.104
11/06/2021 -> 09/07/2021:	203065.691
19/07/2021 -> 17/08/2021:	127236.560
01/03/2022 -> 30/03/2022:	117734.760
26/11/2021 -> 24/12/2021:	266104.849
24/08/2022 -> 22/09/2022:	93825.989
17/09/2021 -> 15/10/2021:	323980.553
22/07/2022 -> 19/08/2022:	-12033.717
06/04/2022 -> 05/05/2022:	156971.428
29/07/2021 -> 27/08/2021:	277042.933
03/06/2022 -> 01/07/2022:	64916.537
10/05/2022 -> 08/06/2022:	262277.281
25/07/2022 -> 23/08/2022:	284806.616

AVERAGED RESULTS FOR 20 RUNS:
|               |     IYW |  

In [69]:
backtester.run_grid_search(
    strategy_params={'quantity': [5, 10]},
    indicator_params={'pairs_ratio': {'WINDOW_SIZE': [10, 30, 50, 100]}},
    cv=20)

> Backtesting the across the following ranges:
Agorithm Parameters {'quantity': [5, 10]}
Indicator Parameters {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 50}, 'pairs_ratio': {'WINDOW_SIZE': [10, 30, 50, 100]}}


Running paramter sweep (cv=20): 100%|██████████| 8/8 [02:00<00:00, 15.05s/it]


Best parameter results:

{'strategy': {'quantity': 5}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 50}, 'pairs_ratio': {'WINDOW_SIZE': 30}}}

Mean ROI:	156909.20364754897
STD ROI:	93090.73333407863

24/05/2022 -> 22/06/2022:	27583.434
22/07/2022 -> 19/08/2022:	-12033.717
05/05/2022 -> 03/06/2022:	171005.386
19/07/2022 -> 17/08/2022:	270222.067
04/03/2022 -> 01/04/2022:	138562.573
28/01/2022 -> 25/02/2022:	59917.444
27/07/2021 -> 25/08/2021:	21183.580
29/09/2021 -> 28/10/2021:	316272.082
22/07/2021 -> 20/08/2021:	201960.922
15/06/2021 -> 14/07/2021:	174107.061
26/07/2022 -> 24/08/2022:	238021.482
09/02/2022 -> 10/03/2022:	162223.991
07/02/2022 -> 08/03/2022:	109613.841
03/08/2022 -> 01/09/2022:	241547.968
18/03/2022 -> 15/04/2022:	220591.896
17/09/2021 -> 15/10/2021:	323980.553
09/04/2021 -> 07/05/2021:	108221.926
20/04/2021 -> 19/05/2021:	128155.390
04/01/2022 -> 02/02/2022:	68501.283
07/09/2021 -> 06/10/2021:	168544.909

AVERAGED RESULTS FOR 20 RUNS:
|      